In [2]:
import networkx as nx
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/Admin/Documents/Projects/Kairos/main/data/GTFS.db')

In [3]:
MG = nx.MultiDiGraph() # initialise Graph

In [4]:
## Add Nodes

In [23]:
stops = pd.read_sql_query('SELECT * FROM gtfs_stops', conn)
stops.head(1)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [6]:
for i, row in stops.iterrows():
    MG.add_node(row['stop_id'], location={'lat': row['stop_lat'], 'long': row['stop_lon']})

## Add Edges

In [8]:
def unicode_to_seconds(a):
    a = map(int, a.split(':'))
    return a[0] * 3600 + a[1] * 60 + a[2]

In [24]:
stops_time = pd.read_sql_query('SELECT * FROM gtfs_stop_times', conn)
stops_time.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200


In [10]:
for i, row in stops_time.iterrows():
    
    try:
        partida, chegada = stops_time.iloc[i], stops_time.iloc[i + 1]
    except:
        break
    if chegada['trip_id'] == partida['trip_id']:
        MG.add_edge(partida['stop_id'], chegada['stop_id'], 
                    attr = {'tempo': chegada['departure_time_seconds']-partida['arrival_time_seconds'],
                      'trip_id': chegada['trip_id']}
        )

## Add frequency

In [13]:
frequencies = pd.read_sql_query('SELECT * FROM gtfs_frequencies', conn)


In [116]:
frequencies.head(30)

,trip_id,start_time,end_time,headway_secs,start_time_seconds,end_time_seconds
0,1012-10-0,03:00:00,03:59:00,1800,10800,14340
1,1012-10-0,04:00:00,04:59:00,420,14400,17940
2,1012-10-0,05:00:00,05:59:00,420,18000,21540
3,1012-10-0,06:00:00,06:59:00,480,21600,25140
4,1012-10-0,07:00:00,07:59:00,420,25200,28740
5,1012-10-0,08:00:00,08:59:00,480,28800,32340
6,1012-10-0,09:00:00,09:59:00,420,32400,35940
7,1012-10-0,10:00:00,10:59:00,480,36000,39540
8,1012-10-0,11:00:00,11:59:00,420,39600,43140
9,1012-10-0,12:00:00,12:59:00,480,43200,46740


In [25]:
stops_time.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
1,1012-10-0,03:42:00,03:42:00,30003039,2,None,None,None,None,None,13320,13320


In [26]:
stops_time_first = stops_time[stops_time['stop_sequence'] == 1]
stops_time_first.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
15,1012-10-1,00:50:00,00:50:00,301729,1,None,None,None,None,None,3000,3000


In [131]:
for i, row in stops_time_first.iterrows():
    print i
    
    lista=[]
    lista.append(row['arrival_time_seconds'])
    while 1:
        
        ultimo = lista[-1]
        

        trip = frequencies[frequencies['trip_id'] == row['trip_id']]
        trip = trip[(trip["start_time_seconds"] <= ultimo) & (trip["end_time_seconds"] >= ultimo)]
        if i == 15:
            print  trip['headway_secs']
        soma = ultimo + trip['headway_secs'].values[0]
            
        if soma < 86340:
            lista.append(soma)
        
        elif soma > 86340:
            
            #print soma
            
            ultimo = soma - 86340
            
            trip = frequencies[frequencies['trip_id'] == row['trip_id']]
            trip = trip[(trip["start_time_seconds"] <= ultimo) & (trip["end_time_seconds"] >= ultimo)]
            print len(trip)
            if len(trip) > 0:
                print 'oi'
                print trip['headway_secs'].values
                lista.append(ultimo + trip['headway_secs'].values[0])
            else:
                break
        else:
            break
    # print lista
    
    #if 'viagens' in MG.node['301729']:
    #    MG.node['301729']['viagens'][row['trip_id']] = lista
    #else:
    #    MG.node['301729']['viagens'] = {row['trip_id']: lista}

0
0
15
21    900
Name: headway_secs, dtype: int64
Series([], Name: headway_secs, dtype: int64)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
lista.append(row['arrival_time_seconds'])
while 1:
    ultimo = lista[-1]
    trip = frequencies[frequencies['trip_id'] == row['trip_id']]
    trip = trip[(trip["start_time_seconds"] < ultimo) & (trip["end_time_seconds"] > ultimo)]
    lista.append(ultimo + trip['headway_secs'])

In [72]:
trip = frequencies[frequencies['trip_id'] == '1012-10-0']
trip[(trip["start_time_seconds"] < ultimo) & (trip["end_time_seconds"] > ultimo)]

,trip_id,start_time,end_time,headway_secs,start_time_seconds,end_time_seconds
2,1012-10-0,05:00:00,05:59:00,420,18000,21540


In [126]:
for i in range(10):
    print i
    if i > 5:
        while 1:
            break

0
1
2
3
4
5
6
7
8
9
